In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as st 
from tqdm import tqdm 
import os
## Change directory to your working directory
os.chdir('D:\Shashi Katteri\Desktop\data sets to work on\Amazon Employee Access\Dataset_10')

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
df=pd.read_csv('Amazon_employee_access.csv')
df.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,target
0,39353,85475,117961,118300,123472,117905,117906,290919,117908,1
1,17183,1540,117961,118343,123125,118536,118536,308574,118539,1
2,36724,14457,118219,118220,117884,117879,267952,19721,117880,1
3,36135,5396,117961,118343,119993,118321,240983,290919,118322,1
4,42680,5905,117929,117930,119569,119323,123932,19793,119325,1


In [3]:
df1=df.astype(str)
df1['target']=df1['target'].astype(int)

In [4]:
#df1['ROLE_ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_TITLE','ROLE_FAMILY'],axis=1)


In [5]:
df1['target']=df1['target'].map({1:0,0:1})

In [6]:
df1['ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_ROLLUP_1','ROLE_ROLLUP_2'],axis=1)

In [7]:
df1.head()

,RESOURCE,MGR_ID,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target,ROLLUP_12
0,39353,85475,123472,117906,117908,0,117961118300
1,17183,1540,123125,118536,118539,0,117961118343
2,36724,14457,117884,267952,117880,0,118219118220
3,36135,5396,119993,240983,118322,0,117961118343
4,42680,5905,119569,123932,119325,0,117929117930


# Combinations with brute force 


In [8]:
from itertools import combinations

cols=list(df1.columns)
cols.remove('target')
d=df1.copy()

for j in combinations(cols,2):
    d[j[0]+'_'+j[1]]=df1[j[0]]+df1[j[1]]
    
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32769 entries, 0 to 32768
Data columns (total 22 columns):
RESOURCE                          32769 non-null object
MGR_ID                            32769 non-null object
ROLE_DEPTNAME                     32769 non-null object
ROLE_FAMILY_DESC                  32769 non-null object
ROLE_CODE                         32769 non-null object
target                            32769 non-null int64
ROLLUP_12                         32769 non-null object
RESOURCE_MGR_ID                   32769 non-null object
RESOURCE_ROLE_DEPTNAME            32769 non-null object
RESOURCE_ROLE_FAMILY_DESC         32769 non-null object
RESOURCE_ROLE_CODE                32769 non-null object
RESOURCE_ROLLUP_12                32769 non-null object
MGR_ID_ROLE_DEPTNAME              32769 non-null object
MGR_ID_ROLE_FAMILY_DESC           32769 non-null object
MGR_ID_ROLE_CODE                  32769 non-null object
MGR_ID_ROLLUP_12                  32769 non-null obj

In [9]:
from lightgbm import LGBMClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split,StratifiedKFold,KFold
from catboost import CatBoostClassifier

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [10]:
def aucroc(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    
    return auc

In [11]:
import warnings
warnings.filterwarnings('ignore')
def kfold_AS_target(train,test,columns,target,weight,folds=10):
    
    columns=list(columns)
    kf=KFold()
    t=train
    v=test
    gm=t[target].mean()
    for col in columns:
        
        for i,j in kf.split(t):
            gb=t.loc[t.index[i]].groupby(col)[target].agg(['count','mean'])
            count=gb['count']
            mean=gb['mean']
            gb['smooth']=(count*mean + weight*gm)/(count+weight)
            t.loc[t.index[j],col+'_enc']=t.loc[t.index[j],col].map(gb['smooth'])
            
        t[col+'_enc']=t[col+'_enc'].fillna(gm)
        
        gb=t.groupby(col)[target].agg(['count','mean'])
        count=gb['count']
        mean=gb['mean']
        gb['smooth']=(count*mean + weight*gm)/(count+weight)
        
        v[col+'_enc']=v[col].map(gb['smooth'])
        v[col+'_enc']=v[col+'_enc'].fillna(gm)
        
    t=t.drop(columns,axis=1)
    v=v.drop(columns,axis=1)

    return t,v
        

In [12]:
def cross_val(algo,d,columns,target):
    sf=StratifiedKFold(n_splits=10)
    model=algo
    fold=0
    fold_mod=[]
    x=df.drop(target,axis=1)
    y=df[target]
    for i,j in sf.split(x,y):

        fold=fold+1

        train=d.iloc[i]
        test=d.iloc[j]
        
        c=columns+[target]
        t,v=kfold_AS_target(train[c],test[c],columns,target,100,folds=10)


        xtr=t.drop(target,axis=1)
        xts=v.drop(target,axis=1)
        ytr=t[target]
        yts=v[target]


        mod=model.fit(xtr,ytr)

        auc=aucroc(mod,xts,yts)
        fold_mod.append(auc)
    return fold_mod


In [13]:
def eliminate(algo,cols,df,target):
    
    high=0

    for i in tqdm(cols):
        c=cols.copy()
        c.remove(i)

       
        auc=cross_val(algo,df,c,target)
        auc_sc=np.array(auc).mean()
        if auc_sc>high:
            high=auc_sc
            r=i

    return high,r


def col_select(algo,step,df,cols,target):
    
    
    
    p=[]
    removed=[]

    high_current=0
    removed_new=[]
    high_new=0
    best=0
   # best_rem=[]
    while high_current-high_new<step:    


        removed=removed+removed_new
        print(len(removed))
        removed_new=[]


        if high_current==0:
            high_new,r=eliminate(algo,cols,df,target)
        else:
            high_current=high_new
        if len(cols)>2:
            
            while high_new>=high_current:
                cols.remove(r)
                removed_new.append(r)
                high_current=high_new

                high_new,r=eliminate(algo,cols,df,target)
                print(high_new)
                if high_new>best:
                    best=high_new
                    best_rem=removed+removed_new+[r]
                    print(best_rem)
        else:
            break

    print('high_auc=',high_current)
    print('auc_one step ahead=',high_new)
    print('removed: ',removed)
    print('len=',len(removed))
    print('best=',best)
    print('best_rem=',best_rem)

In [14]:
rem=['MGR_ID_ROLE_DEPTNAME', 'MGR_ID', 'ROLE_DEPTNAME_ROLLUP_12', 'MGR_ID_ROLLUP_12', 'ROLLUP_12', 'ROLE_CODE_ROLLUP_12',
     'RESOURCE_ROLE_FAMILY_DESC']
d1=d.drop(rem,axis=1)

In [35]:
cols=list(d1.columns)
cols.remove('target')
target='target'

params={'iterations':None,'learning_rate':None,'depth':None,'n_estimators':None,'verbose':False,'eval_metric':'AUC',
        'loss_function':'CrossEntropy'}
iterations=[100,150,200,250]
learning_rate=[0.01,0.001]
depth=[5,6,7]
best=0
#early_stopping_rounds=[5,10,15]

for i in learning_rate:
    params['learning_rate']=i
    for j in tqdm(iterations):
        params['iterations']=j
        for k in depth:
            params['depth']=k
            
            cb=CatBoostClassifier(**params)
            auc=cross_val(cb,d1,cols,target)
            auc_avg=np.array(auc).mean()
            if auc_avg>best:
                best=auc_avg
                bp=params
print(best)
print(bp)




  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [01:50<05:31, 110.61s/it]

 50%|█████     | 2/4 [03:51<03:47, 113.82s/it]

 75%|███████▌  | 3/4 [06:52<02:13, 133.78s/it]

100%|██████████| 4/4 [10:30<00:00, 157.66s/it]


  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [01:52<05:38, 112.86s/it]

 50%|█████     | 2/4 [04:02<03:55, 117.74s/it]

 75%|███████▌  | 3/4 [07:14<02:20, 140.21s/it]

100%|██████████| 4/4 [11:02<00:00, 165.65s/it]


0.8969193305105796
{'iterations': 250, 'learning_rate': 0.001, 'depth': 7, 'n_estimators': None, 'verbose': False, 'eval_metric': 'AUC', 'loss_function': 'CrossEntropy'}


In [24]:
params={'iterations':250,'learning_rate':0.01,'depth':7,'verbose':False,'eval_metric':'AUC',
        'loss_function':'CrossEntropy'}
cb=CatBoostClassifier(**params)
auc=cross_val(cb,d1,cols,target)
auc_avg=np.array(auc).mean()

In [25]:
 auc_avg

0.8969193305105796

In [26]:
cb=CatBoostClassifier(learning_rate=0.01,loss_function='CrossEntropy',eval_metric='AUC',verbose=False)
auc=cross_val(cb,d1,cols,target)
auc_avg=np.array(auc).mean()

In [21]:
auc_avg

0.8934121846393474

In [32]:
params={'learning_rate':0.01,'depth':6,'verbose':False,'eval_metric':'AUC',
        'loss_function':'CrossEntropy'}
cb=CatBoostClassifier(**params)
auc=cross_val(cb,d1,cols,target)
auc_avg=np.array(auc).mean()

In [33]:
auc_avg

0.9009014475905552

[0.8936161828001206,
 0.9023948309893907,
 0.9301348609619287,
 0.9025062656641605,
 0.8858583533664092,
 0.9028302047636096,
 0.8888786592331168,
 0.8997843247574718,
 0.9066970146454572,
 0.8963137787238855]